In [161]:
import pandas as pd
import numpy as np

In [162]:
data= pd.read_csv("soccer_4_14.csv", sep=",", encoding = "ISO-8859-1")


In [165]:
null = data.isnull().sum()
data.shape

print(data.columns)

Index(['id', 'country_id', 'season', 'stage', 'date', 'match_api_id',
       'home_team_api_id', 'away_team_api_id', 'home_team_goal',
       'away_team_goal',
       ...
       'chanceCreationShootingClass1_away',
       'chanceCreationPositioningClass1_away', 'defencePressure1_away',
       'defencePressureClass1_away', 'defenceAggression1_away',
       'defenceAggressionClass1_away', 'defenceTeamWidth1_away',
       'defenceTeamWidthClass1_away', 'defenceDefenderLineClass1_away',
       'outcome'],
      dtype='object', length=103)


In [168]:
filtered0= data.loc[:, (null == 0)]
filtered0 = filtered0[filtered0['League_name'] == "England Premier League"]
#y 1=homewin,0=draw,-1=awaywin
y = filtered0['home_team_goal'] - filtered0['away_team_goal']
y[y>1] = 1
y[y<-1] = 0

#filtering
filtered0 =filtered0.drop(['home_team_goal','away_team_goal',"Goal_Diff_away","Goal_Diff_Home","outcome",
                          'id', 'country_id', 'match_api_id', 'home_team_api_id','away_team_api_id','id1_home_home',
       'team_fifa_api_id_home_home', 'id2_away_away', 'team_api_id1_away_away',
       'team_fifa_api_id1_away_away', 'id5',"date","team_short_name_home_home","team_short_name1_away_away",  
       "id6_away"   ,"team_api_id3_away" ,"date2_away","date1_home" ,"outcome"    ],axis= 1)
# print(filtered0.dtypes)
# filtered
filterlast = filtered0.select_dtypes(exclude=[object])
totransform = filtered0.select_dtypes(include=[object])
dummy = pd.get_dummies(totransform)
final = pd.concat([filterlast,dummy],axis=1)
dataSize = len(y)
ind = np.random.permutation(dataSize)


"80-20 split"
nTest = int(0.20 * dataSize)
# nValid = int(0.10 * dataSize)
# print(dataSize)/
test_indices = ind[:nTest]
# valid_indices = ind[nTest:nTest+nValid]
train_indices = ind[nTest:]

train_set_x = final.iloc[train_indices,:]
train_set_y = y[train_indices]
test_set_x = final.iloc[test_indices,:]
test_set_y = y[test_indices]
# valid_set_x = filtered0.iloc[valid_indices,:]
# valid_set_y = y[valid_indices]


In [142]:
filtered1 = filtered0.filter(['Goal_Diff_away_10.Games' ,'Goal_Diff_Home_10.Games' , 'buildUpPlayPositioningClass_home' , 
    'buildUpPlayPositioningClass1_away' , 'chanceCreationPositioningClass_home' , 
    'chanceCreationPositioningClass1_away' , 'defenceAggression_home' ,
    'chanceCreationShooting_home' , 'chanceCreationShootingClass_home' , 
    'chanceCreationShooting1_away' , 'chanceCreationShootingClass1_away' , 
    'buildUpPlayPassing_home' + 'chanceCreationPassing_home' + 'chanceCreationPassing1_away' , 
    'stage' , 'defenceDefenderLineClass_home' , 'buildUpPlayPassingClass_home' , 
    'chanceCreationCrossing1_away' , 'defenceAggression1_away' + 
    'Country_Name_home'],axis=1)
# filtered1 = filtered0.filter(['Goal_Diff_away_10.Games' ,'Goal_Diff_Home_10.Games' , 'buildUpPlayPositioningClass_home' , 
#     'buildUpPlayPositioningClass1_away' , 'chanceCreationPositioningClass_home' , 
#     'chanceCreationPositioningClass1_away' , 'defenceAggression_home'],axis=1)
# filtered1= filtered0.filter(['VCH' , 'BWH' , 'B365H' , 'LBH' , 'WHH', 'IWH', 'BWA' ,'B365A' , 
#     'VCA' , 'WHA' , 'IWA' , 'LBA' , 'SJA' , 'SJH' ,'PSA', 'PSH', 'GBA', 'BSA', 
#     'BSH' , 'GBH'],axis=1)


filterlast = filtered1.select_dtypes(exclude=[object])
totransform = filtered1.select_dtypes(include=[object])
dummy = pd.get_dummies(totransform)
final = pd.concat([filterlast,dummy],axis=1)
dataSize = len(y)
ind = np.random.permutation(dataSize)


"80-20 split"
nTest = int(0.20 * dataSize)
# nValid = int(0.10 * dataSize)
# print(dataSize)/
test_indices = ind[:nTest]
# valid_indices = ind[nTest:nTest+nValid]
train_indices = ind[nTest:]

train_set_x = final.iloc[train_indices,:]
train_set_y = y[train_indices]
test_set_x = final.iloc[test_indices,:]
test_set_y = y[test_indices]

In [169]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
forest = RandomForestClassifier( n_estimators = 100, criterion='entropy', max_depth=None, min_samples_split=5, 
                                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                                max_leaf_nodes=None, min_impurity_split=1e-07, bootstrap=True, oob_score=False,
                                n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)

# forest = RandomForestClassifier( n_estimators = 1000)

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( train_set_x, train_set_y )
print(np.mean(forest.predict(train_set_x) == train_set_y))
# print (np.mean(cross_val_score(forest, train_set_x.as_matrix(), train_set_y.as_matrix(), cv=10)))
# Test & extract results 
result = forest.predict( test_set_x )
print(np.mean(result==test_set_y))

Fitting a random forest to labeled training data...


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [87]:
np.histogram(y)
                          

(array([5610,    0,    0,    0,    0, 4905,    0,    0,    0, 8835], dtype=int64),
 array([-1. , -0.8, -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ]))

In [33]:

filtered0= data.loc[:, (null == 0)]


In [60]:

filtered0.drop([col for col in filtered0.columns if filtered0[col].dtypes==object],axis=1,inplace=True)


C:\Users\Parit\AppData\Roaming\Python\Python35\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [61]:
filtered0.shape

(19350, 45)

In [78]:
totransform = filtered0.select_dtypes(include=[object])
dummy = pd.get_dummies(totransform)
final = pd.concat([filterlast,dummy],axis=1)

Index(['season', 'team_long_name_home_home', 'team_long_name1_away_away',
       'Country_Name_home', 'League_name', 'buildUpPlaySpeedClass_home',
       'buildUpPlayDribblingClass_home', 'buildUpPlayPassingClass_home',
       'buildUpPlayPositioningClass_home', 'chanceCreationPassingClass_home',
       'chanceCreationCrossingClass_home', 'chanceCreationShootingClass_home',
       'chanceCreationPositioningClass_home', 'defencePressureClass_home',
       'defenceAggressionClass_home', 'defenceTeamWidthClass_home',
       'defenceDefenderLineClass_home', 'buildUpPlaySpeedClass1_away',
       'buildUpPlayDribblingClass1_away', 'buildUpPlayPassingClass1_away',
       'buildUpPlayPositioningClass1_away', 'chanceCreationPassingClass1_away',
       'chanceCreationCrossingClass1_away',
       'chanceCreationShootingClass1_away',
       'chanceCreationPositioningClass1_away', 'defencePressureClass1_away',
       'defenceAggressionClass1_away', 'defenceTeamWidthClass1_away',
       'defenceDefe

In [170]:
train_set_x

,stage,Goal_Diff_Home_10.Games,Goal_Diff_away_10.Games,buildUpPlaySpeed_home,buildUpPlayPassing_home,chanceCreationPassing_home,chanceCreationCrossing_home,chanceCreationShooting_home,defencePressure_home,defenceAggression_home,...,chanceCreationPositioningClass1_away_Organised,defencePressureClass1_away_Deep,defencePressureClass1_away_High,defencePressureClass1_away_Medium,defenceAggressionClass1_away_Double,defenceAggressionClass1_away_Press,defenceTeamWidthClass1_away_Narrow,defenceTeamWidthClass1_away_Normal,defenceTeamWidthClass1_away_Wide,defenceDefenderLineClass1_away_Cover
18499,16,-3,-8,65,40,70,70,55,70,70,...,0,1,0,0,1,0,1,0,0,1
11623,3,10,0,55,70,60,70,55,45,70,...,0,1,0,0,0,1,0,1,0,1
11728,22,2,4,65,70,70,70,55,35,70,...,0,1,0,0,0,1,0,1,0,1
12514,27,14,-17,70,70,70,70,45,70,70,...,1,0,0,1,1,0,0,0,1,1
8720,36,13,-13,70,50,50,65,50,50,50,...,1,0,0,1,1,0,0,1,0,1
13616,38,-3,-6,66,30,30,45,35,30,40,...,1,0,0,1,0,1,0,1,0,1
4311,15,0,4,65,70,70,70,55,35,70,...,1,0,0,1,0,1,0,1,0,1
11671,23,14,4,70,60,56,70,70,30,60,...,0,1,0,0,0,1,0,1,0,1
18729,11,1,-13,65,40,70,70,55,70,70,...,1,0,0,1,0,1,0,1,0,1
4222,25,1,8,70,70,70,70,70,70,70,...,1,0,0,1,0,1,0,1,0,1


In [106]:
8835/(5610+4905+8835)

0.4565891472868217